In [1]:
import sys
sys.path.insert(0, r'../../../quetzal')

In [2]:
from tqdm import tqdm
from shapely import geometry
import pandas as pd
import geopandas as gpd
import time

from imp import reload
from copy import deepcopy
from quetzal.io import road as road_io
reload(road_io)

<module 'quetzal.io.road' from '../../../quetzal\\quetzal\\io\\road.py'>

In [3]:
import json

In [4]:
from quetzal.io.road import *

# UPDATE NODES

In [4]:
with open(r'../../inputs/roads/input.geojson', 'r') as file:
    text = file.read()
road =  json.loads(text)
nodes = gpd.GeoSeries(pd.Series(list(road_io.get_nodes(road))).apply(lambda t: geometry.Point(t)))
nodes.to_file(r'../../inputs/roads/nodes.geojson', driver='GeoJSON')

# SPLIT FEATURES

In [5]:
with open(r'../../inputs/roads/input.geojson', 'r') as file:
    text = file.read()
road =  json.loads(text)
road_io.split_features(road)
with open(r'../../inputs/roads/input.geojson', 'w') as file:
    json.dump(road, file)

# UPDATE TYPES

In [44]:
with open(r'../../inputs/roads/input.geojson', 'r') as file:
    text = file.read()
road =  json.loads(text)
df = gpd.read_file(json.dumps(road))
df['type_bipo'] = df['type_poly']
df.to_file(r'../../inputs/roads/input.geojson', driver='GeoJSON')

# ROADMAP

In [32]:
roadmap = gpd.read_file(r'../../inputs/roads/roads_2040.geojson')

In [33]:
roadmap['poly_scat'] = '9'
roadmap.loc[roadmap['type_poly'] != roadmap['type_scat'], 'poly_scat'] = roadmap['type_poly']

In [34]:
roadmap.to_file(r'../../inputs/roads/roads_2040.geojson', driver='GeoJSON')

# MISCELANEOUS

In [ ]:
stop

In [108]:
df.loc[df['highway'].isin(['track', 'path', 'service', 'unclassified']), 'highway'] = 'residential'
def remove_link(string_or_none):
    try:
        return string_or_none.replace('_link', '')
    except AttributeError:
        return string_or_none
df['highway'] = df['highway'].apply(remove_link)

In [112]:
df['highway'].value_counts()
df.to_file(r'../../inputs/roads/python.geojson', driver='GeoJSON')

In [115]:
df['oneway'] = df['oneway'].fillna(0)

In [122]:
df['type_ref'] = df['highway']
df['type_scat'] = df['highway'].fillna('residential')
df['type_poly'] = df['highway'].fillna('residential')
df['type_bipo'] = df['highway'].fillna('residential')

In [14]:
iterator = tqdm((i for i in range(100)))
textash = 0
for i in iterator:
    with open(r'../../inputs/roads/base.geojson', 'r') as file:
        text = file.read()
    if hash(text) != textash and textash:
        road =  json.loads(text)
        nodes = gpd.GeoSeries(pd.Series(list(get_nodes(road))).apply(lambda t: geometry.Point(t)))
        nodes.to_file(r'../../inputs/roads/base_nodes.geojson', driver='GeoJSON')
        keyboard.press_and_release('F5')
    textash = hash(text)
    time.sleep(1)

100it [03:20,  2.01s/it]
